# Environment Variables

In [1]:
from dotenv import dotenv_values, load_dotenv
config = dotenv_values(".env")
OPENAI_API_KEY={**config}["OPENAI_API_KEY"]

# Chains

## LLMChain

In [3]:
from langchain import LLMChain, OpenAI, PromptTemplate

In [5]:
prompt ='''Eres un asistente virtual experto en {tema} y respondes con una lista de 3 conceptos clave sobre el mismo. Sólo enumeras los tres conceptos'''
template = PromptTemplate.from_template(prompt)

In [13]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY) #modelo
llm_chain = LLMChain(llm=llm, prompt=template) #cadena

In [14]:
llm_chain.predict(tema="Ciencia de datos")

' clave.\n\n1. Análisis de Datos\n2. Aprendizaje Automático\n3. Almacenamiento y Administración de Datos'

## SequantialChain
Unir muchas cadenas secuencialmente que se ejecutan en orden.
En síntesis esta clase permite hacer que la salida de un modelo sirva como entrada para otro.

**Simple Sequential Chain** permite recibir una entrada y generar una salidas

**Sequential Chain** permite recibir múltiples entradas y generar múltiples salidas

In [20]:
from langchain.chains import SequentialChain

In [17]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY) #modelo
prompt ='''Eres un asistente virtual experto en {tema} y respondes con una lista de 3 conceptos clave sobre el mismo. Sólo enumeras los tres conceptos'''
template = PromptTemplate.from_template(prompt)
chains_list = LLMChain(llm=llm, prompt=template, output_key="concepts_list") #cadena

Se arma una cadena que va a recibir la salida de la cadena *chains_list* y lo procesa para generar otro texto

In [18]:
prompt = '''Eres un asistente virtual que recibe una lista de conceptos de un área de conocimiento y debe devolver cuál de esos conceptos es mejor aprender primero. Los conceptos son: {concepts_list}'''
template = PromptTemplate.from_template(prompt)
first_chain = LLMChain(llm=llm,prompt=template,output_key="where_start")

Anteriormente se definieron una cadena lista y una cadena inicio, ahora se unen las cadenas

In [22]:
chains = SequentialChain(chains=[chains_list,first_chain],
                         input_variables=["tema"],
                         output_variables=["concepts_list","where_start"],
                         verbose=True)

In [23]:
chains({"tema":"Física"})



> Entering new SequentialChain chain...

> Finished chain.


{'tema': 'Física',
 'concepts_list': '.\n\n1. Ley de la Gravitación Universal\n2. Ley de la Inercia\n3. Ley de los Seis Grados de Libertad.',
 'where_start': '\n\nLa Ley de la Gravitación Universal es el mejor concepto para aprender primero. Esta ley describe cómo los objetos con masa interactúan entre sí a través de la fuerza de gravedad. La Ley de la Inercia describe cómo los objetos con masa responden a una aceleración, mientras que la Ley de los Seis Grados de Libertad describe cómo los objetos se mueven en tres dimensiones, lo que se conoce como movimiento libre.'}

### Sequencial Chains examples

In [29]:
#LLMMathChain requires the numexpr package. Please install it with `pip install numexpr`
from langchain import LLMMathChain
math_chain = LLMMathChain(llm=llm, verbose=True)
math_chain.run("Cuánto es 12462x81 al cuadrado x 5")

c:\Users\usuario\Desktop\Code\TG\LangChain\venv\lib\site-packages\langchain\chains\llm_math\base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
Cuánto es 12462x81 al cuadrado x 5```text
12462*81**2 * 5
```
...numexpr.evaluate("12462*81**2 * 5")...

Answer: 408815910
> Finished chain.


'Answer: 408815910'

In [ ]:
from langchain.chains import TransformChain

def delete_line_breaks(input):
    text= input["texto"]
    return {"clean_text":text.replace("\n"," ")}

transform_chain = TransformChain(input_variables=["texto"],
                                 output_variables=["clean_text"],
                                 transform=delete_line_breaks)

prompt("\nEste es un\n texto\n con saltos \nde línea, \npapá")